In [2]:
from os import walk

fitsfiles = [] 
for (dirpath, dirnames, filenames) in walk('./fitsfiles'):
    for file in filenames:    
        if(file.endswith('.fits')):
            if('pfss_intoout' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_IO'))
            elif('pfss_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'PFSS_OI'))
            elif('scs_outtoin' in file):
                fitsfiles.append((dirpath + '/' + file, 'SCS_OI'))
    break

### Defintion from OpenSpace function FieldlinesState::saveStateToOsfls
    File is structured like this: (for version 0)
    0. int                    - version number of binary state file! (in case something needs to be altered in the future, then increase CurrentVersion)
    1. double                 - _triggerTime
    2. int                    - _model
    3. bool                   - _isMorphable
    4. size_t                 - Number of lines in the state  == _lineStart.size() == _lineCount.size()
    5. size_t                 - Total number of vertex points  == _vertexPositions.size() == _extraQuantities.size()
    6. size_t                 - Number of extra quantites  == _extraQuantities.size() == _extraQuantityNames.size()
    7. site_t                 - Number of total bytes that ALL _extraQuantityNames  consists of (Each such name is stored as a c_str which means it ends with the null char '\0' )
    8. std::vector<GLint>     - _lineStart
    9. std::vector<GLsizei>   - _lineCount
    10. std::vector<glm::vec3> - _vertexPositions
    11. std::vector<float>     - _extraQuantities
    12. array of c_str         - Strings naming the extra quantities (elements of _extraQuantityNames). Each string ends with null char '\0'

In [7]:
import math, numpy

def sph2cart(coord):
    return [
         coord[2] * math.sin(coord[1]) * math.cos(coord[0]), 
         coord[2] * math.sin(coord[1]) * math.sin(coord[0]),
         coord[2] * math.cos(coord[1])
    ]

def coord2datarow(coord):
    sph = sph2cart(coord[0:3])
    sph.append(coord[5])
    return [numpy.float32(x) for x in sph] 

In [8]:
from enum import Enum
class Model(Enum):
    Batsrus = 0
    Enlil = 1
    Pfss = 2
    Wsa = 3
    Invalid = 5

In [9]:
def obstimeToJ2000(ot):
    timefits = ot[0:4] + '-' + ot[5:7] + '-' + ot[8:10] + 'T'
    timefits += ot[11:13] + ot[14:17] + ot[18:21] + '.000'
    pathSafeTimeString = timefits.replace(':', '-')
    time = Time(timefits, format='fits')
    y2000 = Time(2000, format='jyear')
    jdaysdelta = time.jd - y2000.jd
    jsecs = jdaysdelta*60*60*24
    return [jsecs, pathSafeTimeString]

In [20]:
from astropy.io import fits
from astropy.time import Time
import struct, ctypes, math

def toOsfls(filename, modelname, indices,  typename, xq_filename = ""):
    fl_fits = fits.open(filename)
    fl_data = fl_fits[0].data
    fl_fits.close()
    
    # read file for extra quantity
    if(modelname != 'PFSS_IO'):
        vel_fits = fits.open(xq_filename)
        vel_data = vel_fits[0].data
        vel_data = vel_data[1] # the second layer has the wind speed
        vel_fits.close()
        
    
    
    versionNumber = 0
    [triggerTime, pathSafeTimeString] = obstimeToJ2000(fl_fits[0].header['OBSTIME'])
    #print("TriggerTime: ",triggerTime)
    print("pathSafeTimeString: ",pathSafeTimeString)
    fileName = pathSafeTimeString + '.osfls'

    model = Model.Wsa.value
    isMorphable = False

    nVert = 0
    lineStart = []
    lineCount = []
    vertexPositions = []
    extraQuantities = []

    for i in indices:
        points = [coord2datarow(pt) for pt in fl_data[i] if pt[0] > -900] 
        if (len(points) < 2): continue
        lineStart.append(nVert)
        nVert += len(points)
        lineCount.append(len(points))
        [vertexPositions.extend(pt[0:3]) for pt in points] # extend to unfold elements
        if(modelname == 'PFSS_IO'):
            [extraQuantities.append(pt[3]) for pt in points]
        else:     
            velocity = vel_data[math.floor(i/180)][i%180]
            if(points[0][3] < 0): velocity = -1*velocity
            xtra = [velocity]*len(points) # same value for all points
            extraQuantities.extend(xtra)
    
    nLines = len(lineStart)

    nExtras = 1
    if(modelname == 'PFSS_IO'):
        extraQuantityNames = ['b-field radius \0']
    else: 
        extraQuantityNames = ['wind speed with polarity \0']
    nStringBytes = sum([len(s) for s in extraQuantityNames])
    allNamesInOne = ''
    for s in extraQuantityNames:
        allNamesInOne += s
    
    # Prepare data for writing to binary. Using Struct and pack
    typestr = '= i d i ? Q Q Q Q %sl %sL %sf %sf %ss' % (nLines, nLines, 3*nVert, nVert, nStringBytes)
    struct_to_write = struct.Struct(typestr)
    #print('Format string  :', struct_to_write.format)
    print('Uses           :', struct_to_write.size, 'bytes')
    values_to_write = (versionNumber, triggerTime, model, isMorphable, nLines, nVert, nExtras, nStringBytes)
    values_to_write += (*lineStart, *lineCount, *vertexPositions, *extraQuantities, allNamesInOne.encode('utf-8'))
    
    buffer = ctypes.create_string_buffer(struct_to_write.size)    
    struct_to_write.pack_into(buffer, 0, *values_to_write)
    
    fout = open('./' + modelname + '/' + typename + '_' + fileName, 'wb')
    fout.write(buffer)
    fout.close()


### Make into osfls format - every nth

In [11]:
def everynth(step):
    return (range(0,16200, step), 'step' + str(step))

In [ ]:
import time
start_time = time.time()

indices = everynth(25)

for fitsfile in fitsfiles:
    toOsfls(fitsfile[0], fitsfile[1], indices[0], indices[1])
    print('Finished converting {} after {} seconds.'.format(fitsfile[0],time.time()-start_time))
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))

#### Only PFSS_IO

In [18]:
import time
start_time = time.time()
# relies heavily on having a sorted list of files for each timestep
# so that pfss_io goes first and picks out the lines
for timestamp in fitsfilesdates:
    fitsfilesdates[timestamp].sort()
    pfss_io = fitsfilesdates[timestamp][0]
    indices = everynth(13)
    toOsfls(pfss_io[0], pfss_io[1], indices[0], indices[1], wsafilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(pfss_io[1],time.time()-start_time))
    
        
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))


pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 354816 bytes
Finished converting PFSS_IO after 0.8480010032653809 seconds.
pathSafeTimeString:  2019-05-02T12-00-00.000
Uses           : 346232 bytes
Finished converting PFSS_IO after 1.6761369705200195 seconds.
pathSafeTimeString:  2019-05-02T18-00-00.000
Uses           : 359056 bytes
Finished converting PFSS_IO after 2.592864990234375 seconds.
pathSafeTimeString:  2019-05-02T14-00-00.000
Uses           : 351032 bytes
Finished converting PFSS_IO after 3.518928050994873 seconds.
pathSafeTimeString:  2019-05-02T16-00-00.000
Uses           : 353016 bytes
Finished converting PFSS_IO after 4.318600177764893 seconds.
pathSafeTimeString:  2019-05-02T20-00-00.000
Uses           : 360432 bytes
Finished converting PFSS_IO after 5.060018301010132 seconds.
pathSafeTimeString:  2019-05-02T10-00-00.000
Uses           : 352000 bytes
Finished converting PFSS_IO after 5.903692960739136 seconds.
Execution time for type step13: 5.903847217559

### Pick closed field lines from PFSS_IO
By removing the open ones. Opens ones are defined by their first and last point having the same polarity.

In [19]:
def pickClosed(filename):
    fl_fits = fits.open(filename)
    fl_data = fl_fits[0].data

    indices_to_save = []

    for i in range(16200):
        b_radii = [pt[5] for pt in fl_data[i] if pt[0] > -900] 
        if (len(b_radii) < 2): continue
        first_b_radius = b_radii[0]
        last_b_radius = b_radii[-1]
        if(first_b_radius*last_b_radius < 0): #if product is negative/opposite signs
            indices_to_save.append(i)
    return indices_to_save

In [8]:
# make a pfss_io closed lines sparser
def make_sparser(indices, step):
    new_indices = indices[::3]
    return [new_indices, 'closed_step' + str(step)]

### Pick boundary lines

In [9]:
import math

def boundaryLines(filename_io, filename_oi, indices_closed):
    fl_io = (fits.open(filename_io))[0].data
    fl_oi = (fits.open(filename_oi))[0].data

    threshold = math.sqrt(2)
    # boundary_lines_io = [] do we want these?
    boundary_lines_oi = []
    io_last_phi = {}
    io_last_theta = {}
    io_first_phi = {}
    io_first_theta = {}

    for j in indices_closed:
        last = -1 # find last index
        for point in fl_io[j]:
            if (point[0] > -900):
                last += 1
            else: break
        # get the first and last coordinates for in-to-out (both on surface)
        io_last_phi[j] = math.degrees(fl_io[j][last][0])
        io_last_theta[j] = math.degrees(fl_io[j][last][1])
        io_first_phi[j] = math.degrees(fl_io[j][0][0])
        io_first_theta[j] = math.degrees(fl_io[j][0][1])

    for i in range(16200):
        last = -1
        for point in fl_oi[i]:
            if (point[0] > -900):
                last += 1
            else: break
        if (last < 2): continue
        # get the last coordinates for out-to-in (the ones on the surface)
        oi_last_phi = math.degrees(fl_oi[i][last][0])
        oi_last_theta = math.degrees(fl_oi[i][last][1])
        for j in indices_closed:
            # calculate distances and compare
            dist_last_phi = abs(io_last_phi[j] - oi_last_phi)
            dist_last_theta = abs(io_last_theta[j] - oi_last_theta)
            dist_first_phi = abs(io_first_phi[j] - oi_last_phi)
            dist_first_theta = abs(io_first_theta[j] - oi_last_theta)
            if(dist_last_phi < threshold and dist_last_theta < threshold):
                #boundary_lines_io.append(j)
                boundary_lines_oi.append(i)
                break
            if(dist_first_phi < threshold and dist_first_theta < threshold):
                #boundary_lines_io.append(j)
                boundary_lines_oi.append(i)
                break
    return [boundary_lines_oi, 'boundary']
    

### Populate file list based on timestamp

In [14]:
fitsfilesdates = {} 

for (dirpath, dirnames, filenames) in walk('./fitsfiles'):
    for file in filenames:    
        if(file.endswith('.fits')):
            date = file[0:12]
            if(date not in fitsfilesdates):
                fitsfilesdates[date] = []
            if('pfss_intoout' in file):
                fitsfilesdates[date].append((dirpath + '/' + file, 'PFSS_IO'))
            elif('pfss_outtoin' in file):
                fitsfilesdates[date].append((dirpath + '/' + file, 'PFSS_OI'))
            elif('scs_outtoin' in file):
                fitsfilesdates[date].append((dirpath + '/' + file, 'SCS_OI'))
    break

In [3]:
velfilesdates = {}
wsafilesdates = {}
for (dirpath, dirnames, filenames) in walk('./fitsfiles_images'):
    for file in filenames:    
        if('vel' in file):
            date = file[4:16]
            velfilesdates[date] = dirpath + '/' + file
        if('wsa' in file):
            date = file[4:16]
            wsafilesdates[date] = dirpath + '/' + file

### Fill out those boundary lines
With some sparse lines in between

In [12]:
def fillOut(boundary_indices, step):
    output = []
    previous_index = 0
    for index in boundary_indices:
        subrange = list(range(previous_index, index, step))
        output.extend(subrange)
        if(output[-1] != index):    
            output.append(index)
        previous_index = index
    return [output, 'boundary_filled']


### Make into osfls - closed lines and boundary lines with fillers

In [72]:
import time
start_time = time.time()
# relies heavily on having a sorted list of files for each timestep
# so that pfss_io goes first and picks out the lines
for timestamp in fitsfilesdates:
    fitsfilesdates[timestamp].sort()
    pfss_io = fitsfilesdates[timestamp][0]
    pfss_oi = fitsfilesdates[timestamp][1]
    scs_oi = fitsfilesdates[timestamp][2]
    indices_closed_lines = pickClosed(pfss_io[0])
    print('Picked closed lines for {} after {} seconds.'.format(timestamp,time.time()-start_time))
    indices = make_sparser(indices_closed_lines, 3) # change here for sparser
    toOsfls(pfss_io[0], pfss_io[1], indices[0], indices[1], wsafilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(pfss_io[1],time.time()-start_time))
    indices = boundaryLines(pfss_io[0], pfss_oi[0], indices_closed_lines)
    indices = fillOut(indices[0], 25) # change step here for sparser
    print('Picked boundary lines after {} seconds.'.format(time.time()-start_time))
    toOsfls(pfss_oi[0], pfss_oi[1], indices[0], indices[1], velfilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(pfss_oi[1],time.time()-start_time))
    toOsfls(scs_oi[0], scs_oi[1], indices[0], indices[1], velfilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(scs_oi[1],time.time()-start_time))
        
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))


Picked closed lines for 201905020800 after 1.5125329494476318 seconds.
Finished converting PFSS_IO after 1.5126447677612305 seconds.
Picked boundary lines after 51.77428603172302 seconds.
(90, 180)
pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 7573171 bytes
Finished converting PFSS_OI after 54.353524923324585 seconds.
(90, 180)
pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 9476243 bytes
Finished converting SCS_OI after 58.040857791900635 seconds.
Picked closed lines for 201905021200 after 61.26558303833008 seconds.
Finished converting PFSS_IO after 61.26574778556824 seconds.
Picked boundary lines after 118.85041189193726 seconds.
(90, 180)
pathSafeTimeString:  2019-05-02T12-00-00.000
Uses           : 7403011 bytes
Finished converting PFSS_OI after 121.37582683563232 seconds.
(90, 180)
pathSafeTimeString:  2019-05-02T12-00-00.000
Uses           : 9327203 bytes
Finished converting SCS_OI after 125.12558197975159 seconds.
Picked closed lines for 20190502160

### Pick the last 540 (sun-earth connection)
And turn OI sets to osfls

In [15]:
import time
start_time = time.time()

indices = [range(16200,16740), 'sun_earth']

for timestamp in fitsfilesdates:
    fitsfilesdates[timestamp].sort()
    pfss_oi = fitsfilesdates[timestamp][1]
    scs_oi = fitsfilesdates[timestamp][2]
    toOsfls(pfss_oi[0], pfss_oi[1], indices[0], indices[1])
    print('Finished converting {} after {} seconds.'.format(pfss_oi[1],time.time()-start_time))
    toOsfls(scs_oi[0], scs_oi[1], indices[0], indices[1])
    print('Finished converting {} after {} seconds.'.format(scs_oi[1],time.time()-start_time))
        
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))

pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 1588363 bytes
Finished converting PFSS_OI after 0.6377420425415039 seconds.
pathSafeTimeString:  2019-05-02T08-00-00.000
Uses           : 1908619 bytes
Finished converting SCS_OI after 1.3328490257263184 seconds.
pathSafeTimeString:  2019-05-02T12-00-00.000
Uses           : 1594235 bytes
Finished converting PFSS_OI after 1.9223661422729492 seconds.
pathSafeTimeString:  2019-05-02T12-00-00.000
Uses           : 1941995 bytes
Finished converting SCS_OI after 2.6462481021881104 seconds.
pathSafeTimeString:  2019-05-02T16-00-00.000
Uses           : 1594267 bytes
Finished converting PFSS_OI after 3.2427361011505127 seconds.
pathSafeTimeString:  2019-05-02T16-00-00.000
Uses           : 1945979 bytes
Finished converting SCS_OI after 3.998926877975464 seconds.
pathSafeTimeString:  2019-05-02T20-00-00.000
Uses           : 1600347 bytes
Finished converting PFSS_OI after 4.5705530643463135 seconds.
pathSafeTimeString:  2019-05-02T20-00-